In [2]:
# from google.colab import drive
# drive.mount("/content/drive")
# TensorFlow and tf.keras
import tensorflow as tf

from tensorflow import keras

# Helper libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds

import urllib3

urllib3.disable_warnings()


print(tf.__version__)
print(pd.__version__)
print(np.__version__)
tf.config.list_physical_devices("GPU")


import tensorflow as tf
from tensorflow.keras.layers import (
    Conv2D,
    BatchNormalization,
    Activation,
    MaxPooling2D,
    Add,
    AveragePooling2D,
    Flatten,
    Dense,
    Input,
)
from tensorflow.keras.models import Model


def residual_block(x, filters, kernel_size=3, stride=1, conv_shortcut=True, name=None):
    bn_axis = 3  # for 'channels_last' data format
    if conv_shortcut:
        shortcut = Conv2D(filters, 1, strides=stride, name=name + "_0_conv")(x)
        shortcut = BatchNormalization(axis=bn_axis, name=name + "_0_bn")(shortcut)
    else:
        shortcut = x

    x = Conv2D(
        filters, kernel_size, padding="SAME", strides=stride, name=name + "_1_conv"
    )(x)
    x = BatchNormalization(axis=bn_axis, name=name + "_1_bn")(x)
    x = Activation("relu", name=name + "_1_relu")(x)

    x = Conv2D(filters, kernel_size, padding="SAME", name=name + "_2_conv")(x)
    x = BatchNormalization(axis=bn_axis, name=name + "_2_bn")(x)

    x = Add(name=name + "_add")([shortcut, x])
    x = Activation("relu", name=name + "_out")(x)
    return x


def build_resnet(
    input_shape=(32, 32, 3),
    is_50=False,
):

    num_classes = 10

    inputs = Input(shape=input_shape, name="input_layer")
    x = Conv2D(64, 3, strides=2, padding="same", name="conv1")(inputs)
    x = BatchNormalization(axis=3, name="bn_conv1")(x)
    x = Activation("relu", name="conv1_relu")(x)
    x = MaxPooling2D(3, strides=2, padding="same", name="pool1")(x)

    x = residual_block(x, 64, conv_shortcut=False, name="conv2_1")
    x = residual_block(x, 64, conv_shortcut=False, name="conv2_2")
    x = residual_block(x, 64, conv_shortcut=False, name="conv2_3")

    x = residual_block(x, 128, stride=2, name="conv3_1")
    x = residual_block(x, 128, conv_shortcut=False, name="conv3_2")
    x = residual_block(x, 128, conv_shortcut=False, name="conv3_3")
    x = residual_block(x, 128, conv_shortcut=False, name="conv3_4")

    x = residual_block(x, 256, stride=2, name="conv4_1")
    x = residual_block(x, 256, conv_shortcut=False, name="conv4_2")
    x = residual_block(x, 256, conv_shortcut=False, name="conv4_3")
    x = residual_block(x, 256, conv_shortcut=False, name="conv4_4")
    x = residual_block(x, 256, conv_shortcut=False, name="conv4_5")
    x = residual_block(x, 256, conv_shortcut=False, name="conv4_6")

    x = residual_block(x, 512, stride=2, name="conv5_1")
    x = residual_block(x, 512, conv_shortcut=False, name="conv5_2")
    x = residual_block(x, 512, conv_shortcut=False, name="conv5_3")

    x = AveragePooling2D(pool_size=(4, 4), name="avg_pool")(x)
    x = Flatten(name="flatten")(x)
    outputs = Dense(num_classes, activation="softmax", name="fc1000")(x)

    model = Model(inputs, outputs, name="resnet34")
    return model


resnet_34 = build_resnet(input_shape=(32, 32, 3), is_50=False)
resnet_34.summary()

resnet_50 = build_resnet(input_shape=(32, 32, 3), is_50=True)
resnet_50.summary()


2.16.2
2.2.2
1.26.4


Model: "resnet34"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1 (Conv2D)      │ (None, 16, 16,    │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_conv1            │ (None, 16, 16,    │        256 │ conv1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 16, 16,    │          0 │ bn_conv1[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 8, 8, 64)  │          0 │ conv1_relu[0][0]  │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_1_1_conv      │ (None, 8, 8, 64)  │     36,928 │ pool1[0][0]       │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_1_1_bn        │ (None, 8, 8, 64)  │        256 │ conv2_1_1_conv[0… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_1_1_relu      │ (None, 8, 8, 64)  │          0 │ conv2_1_1_bn[0][… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_1_2_conv      │ (None, 8, 8, 64)  │     36,928 │ conv2_1_1_relu[0… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_1_2_bn        │ (None, 8, 8, 64)  │        256 │ conv2_1_2_conv[0… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_1_add (Add)   │ (None, 8, 8, 64)  │          0 │ pool1[0][0],      │
│                     │                   │            │ conv2_1_2_bn[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_1_out         │ (None, 8, 8, 64)  │          0 │ conv2_1_add[0][0] │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_2_1_conv      │ (None, 8, 8, 64)  │     36,928 │ conv2_1_out[0][0] │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_2_1_bn        │ (None, 8, 8, 64)  │        256 │ conv2_2_1_conv[0… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_2_1_relu      │ (None, 8, 8, 64)  │          0 │ conv2_2_1_bn[0][… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_2_2_conv      │ (None, 8, 8, 64)  │     36,928 │ conv2_2_1_relu[0… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_2_2_bn        │ (None, 8, 8, 64)  │        256 │ conv2_2_2_conv[0

 Total params: 21,302,538 (81.26 MB)

 Trainable params: 21,285,514 (81.20 MB)

 Non-trainable params: 17,024 (66.50 KB)